In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('D:\\Загрузки\\ml-latest\\movies_ratings.csv')

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)
df

,uId,iId,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0
...,...,...,...
1570727,15476.0,Someone (2016),2.0
1570728,11655.0,The Fencing Master (1992),3.0
1570729,4871.0,Empowered (2018),2.0
1570730,4226.0,Raazi (2018),4.0


In [4]:
# 1 вариант - блендинг из функций: SVD, KNeighbors
# В результате набюдений было принято решение не использовать SVD, так как он сильно лучше предсказывает остальных алгоритмов


In [252]:
df_e = df.sample(frac=1, replace=True).reset_index(drop=True)
data = df_e.loc[:150000, :]

In [275]:
data

,uId,iId,rating
0,7036.0,Pitch Black (2000),3.0
1,14346.0,Night of the Living Dead (1968),4.0
2,9917.0,Galaxy Quest (1999),4.0
3,9916.0,"Flintstones, The (1994)",2.5
4,11201.0,Hook (1991),3.5
...,...,...,...
149996,15259.0,Philadelphia (1993),3.0
149997,14061.0,Annie (1982),4.0
149998,15645.0,Sherlock Holmes: A Game of Shadows (2011),4.5
149999,9498.0,The Martian (2015),4.5


In [254]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import NormalPredictor, KNNWithMeans
# from implicit.als import AlternatingLeastSquares
# import scipy.sparse as sparse

In [255]:
import os
os.environ['MKL_NUM_THREADS'] = '1'

In [276]:
# als = AlternatingLeastSquares(factors=20)
k_model = KNNBasic(k=50, sim_options={'name': 'cosine', 'user_based': True})
norm_p = NormalPredictor()
k_means = KNNWithMeans()
svd = SVD()

In [277]:
reader = Reader(rating_scale=(0.5, 5))
dataset = Dataset.load_from_df(data, reader)
train_set, test_set = train_test_split(dataset, test_size=0.2)

In [279]:
# Точность метода Ближайших К соседей
k_model.fit(train_set)
pred_k = k_model.test(test_set)
accuracy.rmse(pred_k, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0637


1.0636844250777784

In [280]:
# Точность метода разброса по нормальному распределению
norm_p.fit(train_set)
p_pred = norm_p.test(test_set)
accuracy.rmse(p_pred, verbose=True)

RMSE: 1.4766


1.4765534072209208

In [281]:
# Точность метода Ближайших К соседей с учетом средних
k_means.fit(train_set)
k_pred = k_means.test(test_set)
accuracy.rmse(k_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0032


1.0031780274321886

##### Результаты предсказаний каждого метода занесем в списки, для решения задачи линейной регрессии

In [282]:
k = []
s = []
l = []
y = []
for _ in test_set:
    k.append(k_model.predict(uid=_[0], iid=_[1]))
    s.append(norm_p.predict(uid=_[0], iid=_[1]))
    l.append(k_means.predict(uid=_[0], iid=_[1])) 
    
for c in range(len(test_set)):    
    y.append(test_set[c][2])

In [287]:
print(k_model.predict(uid=test_set[1][0], iid=test_set[1][1]))
print(norm_p.predict(uid=test_set[1][0], iid=test_set[1][1]))
print(k_means.predict(uid=test_set[1][0], iid=test_set[1][1]))

user: 589.0      item: Dead Poets Society (1989) r_ui = None   est = 3.91   {'actual_k': 27, 'was_impossible': False}
user: 589.0      item: Dead Poets Society (1989) r_ui = None   est = 2.06   {'was_impossible': False}
user: 589.0      item: Dead Poets Society (1989) r_ui = None   est = 4.22   {'actual_k': 27, 'was_impossible': False}


In [288]:
array = []
for i in range(len(k)):
    array.append((k[i][3], s[i][3], l[i][3]))

In [301]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import PolynomialFeatures

# pipeline = Pipeline([('poly', PolynomialFeatures()), ('linear', LinearRegression()), ('ridge', Ridge())])

model = Ridge(alpha=1000)
model.fit(X=array, y=y)


Ridge(alpha=1000)

Для нахождения коэфицеентов применена модель Ridge(), так как на лицо разбалансировка в итоговых коэфицеентах  
и на мой взгляд необходима регуляризация

In [302]:
print(model.coef_)
print(model.intercept_)

[0.0675363  0.00335953 0.57849684]
1.2147977232952254


In [303]:
preds = [model.predict(np.array(array[h]).reshape(1, -1)) for h in range(len(array))]

In [304]:
mean_squared_error(y, preds)

0.9274817111425432

Итоговая ошибка, действительно получилась меньше, чем у каждого метода отдельно